In [172]:
import pandas as pd
pd.set_option('max_colwidth', 20)
import re

In [2]:
df = pd.read_csv("Openrice_all_lunch.csv", index_col = 0)

In [3]:
df.head()

,District,Cuisine,Reviews,Restaurant_Name,Rating,Cost,Wait_Time,Restuaruant_Link
0,Admiralty,American,"I am a burger lover.Today, I need to work in A...",Shake Shack,4.8,140,0.0,www.openrice.com/en/hongkong/restaurant/admira...
1,Admiralty,American,Its a very famouly burger place in HK. Its qui...,Shake Shack,4.5,100,20.0,www.openrice.com/en/hongkong/restaurant/admira...
2,Admiralty,American,I have been to Commissary a couple of times fo...,Commissary,NaN,200,0.0,www.openrice.com/en/hongkong/restaurant/admira...
3,Admiralty,American,Commissary really has the right vibe for us. T...,Commissary,NaN,250,0.0,www.openrice.com/en/hongkong/restaurant/admira...
4,Admiralty,American,Love the ambiance and comfort food. Cozy space...,Commissary,3.7,170,0.0,www.openrice.com/en/hongkong/restaurant/admira...


### Clean missing values

In [5]:
df_c = df.dropna(subset=["Rating","Restaurant_Name"])

In [6]:
df_c.isna().sum()

District              0
Cuisine               0
Reviews               0
Restaurant_Name       0
Rating                0
Cost                  0
Wait_Time           450
Restuaruant_Link      0
dtype: int64

### Add Restaurant ID

In [7]:
# df[newIdName] = (df['LastName'] + '_' + df['FirstName']).astype('category').cat.codes
# https://stackoverflow.com/questions/45685254/q-pandas-how-to-efficiently-assign-unique-id-to-individuals-with-multiple-ent

# df_a = df_c[["District","Reviews", "Restaurant_Name","Rating"]]
# df_a.head()

In [8]:
# df_a["Restaurant_ID"] = (df_a["Restaurant_Name"] + "_" + df_a["District"]).astype("category").cat.codes

In [9]:
# df_a.head(5)

In [10]:
df_b = df_c

In [11]:
# df_c_cut["Restaurant_ID"].nunique()

In [12]:
# df_a["Restaurant_ID"].nunique()

In [13]:
df_b["Restaurant_ID"] = (df_b["Restaurant_Name"] + "_" + df_b["District"]).astype("category").cat.codes

/Users/makhy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
df_b.head()

,District,Cuisine,Reviews,Restaurant_Name,Rating,Cost,Wait_Time,Restuaruant_Link,Restaurant_ID
0,Admiralty,American,"I am a burger lover.Today, I need to work in A...",Shake Shack,4.8,140,0.0,www.openrice.com/en/hongkong/restaurant/admira...,609
1,Admiralty,American,Its a very famouly burger place in HK. Its qui...,Shake Shack,4.5,100,20.0,www.openrice.com/en/hongkong/restaurant/admira...,609
4,Admiralty,American,Love the ambiance and comfort food. Cozy space...,Commissary,3.7,170,0.0,www.openrice.com/en/hongkong/restaurant/admira...,138
5,Admiralty,American,I'm an old fashioned person who enjoys old fas...,Ruth's Chris Steak House,3.8,350,NaN,www.openrice.com/en/hongkong/restaurant/admira...,584
6,Admiralty,American,"Our China colleague came, and we dined out tog...",Commissary,3.2,180,0.0,www.openrice.com/en/hongkong/restaurant/admira...,138


### Group by Restaurant ID

In [201]:
# https://stackoverflow.com/questions/14529838/apply-multiple-functions-to-multiple-groupby-columns

def f(x):
    d = {}
    d["Restaurant_Name"] = x["Restaurant_Name"].unique()[0]
    d["District"] = x["District"].unique()[0]
    d["Cuisine"] = x["Cuisine"].unique()[0]
    d["Rating_Average"] = round(x["Rating"].mean(),1)
    d["Cost_Average"] = round(x["Cost"].mean(),1)
    d["Wait_Average"] = round(x["Wait_Time"].mean(),0)
#     d["Reviews"] = x["Reviews"].agg(lambda col: ''.join(col)).reset_index()
#     d["Reviews"] = x["Reviews"].agg("sum")
    d["Reviews"] = str(x["Reviews"].agg(lambda x: x.tolist())).strip()[2:-1]
#     d["Reviews"] = str(x["Reviews"].agg(''.join))
    d["Restaurant_Link"] = x["Restuaruant_Link"].unique()[0]
#     d["Count"] = x["Reviews"].value_counts()
    return pd.Series(d, index = ["Restaurant_Name","District", "Cuisine",
                                 "Rating_Average","Cost_Average", 
                                 "Wait_Average","Reviews",
                                 "Restaurant_Link"])

In [202]:
# df_all = df_b.groupby("Restaurant_ID").apply(f)
df_all_new = df_b.groupby("Restaurant_ID").apply(f)

In [203]:
df_all_new.head(20)

,Restaurant_Name,District,Cuisine,Rating_Average,Cost_Average,Wait_Average,Reviews,Restaurant_Link
Restaurant_ID,,,,,,,,
0,CHIRASHI,Wan Chai,Japanese,3.7,488.0,0.0,0Chirashi is a p...,www.openrice.com...
1,X,Causeway Bay,Chinese,3.6,60.0,0.0,There are a numb...,www.openrice.com...
2,10 Shanghai,Causeway Bay,Chinese,3.2,120.0,NaN,1957 & Company (...,www.openrice.com...
3,1935 Restaurant,Central,Chinese,4.8,270.0,NaN,The food is deli...,www.openrice.com...
4,1963,Wan Chai,Chinese,3.6,113.3,2.0,00This shop in W...,www.openrice.com...
5,2/3 Dolci,Causeway Bay,Italian,3.5,200.0,NaN,Dolci (n.) - Swe...,www.openrice.com...
6,298 Nikuya Room,Central,Japanese,4.6,315.0,0.0,Suddenly craved ...,www.openrice.com...
7,3.6.9. Restauran...,Wan Chai,Chinese,3.1,55.0,0.0,As boyfriend and...,www.openrice.com...
8,33. Kyoto,Wan Chai,Japanese,3.8,58.5,0.0,With so many bur...,www.openrice.com...


In [162]:
# text_test_1 = df_all["Reviews_1"][12]

In [205]:
text_test = df_all_new["Reviews"][12]

In [128]:
df_all["Restaurant_Link"][1]

'www.openrice.com/en/hongkong/restaurant/causeway-bay-%E6%A3%8B%E5%93%A5%E7%87%92%E9%B5%9D-x-%E7%82%AF%E8%A8%98%E9%A4%90%E5%AE%A4/588528?tc=rvw2&con=rvw'

In [206]:
print(text_test)

It was our second visit. Our first visit about a few years ago hadn't given us a very impressive impression but we had just ordered a few items from their a la carte menu so we thought maybe it was a time for a re-visit and for trying their set lunch menu.A window-side table had been reserved quite a while ago during weekday lunch.Lunch Menu1Lunch MenuLunch Menu0Lunch MenuLunch Menu0Lunch MenuLunch Menu0Lunch MenuIt seemed that after a few years their a la carte menu looked pretty much the same. I still remembered that we had tried Marinated Tuna Tartare which was quite good while their Baby Lamb from Aveyron and Short Rib & Beef Tenderloin were nothing outstanding especially the former.Set Lunch Menu1Set Lunch MenuThere were two lunch sets: 4 courses including appetizer, pasta, main and dessert or cheese at $1080+10% and 3 courses with either pasta or main at your choice and everything else the same at $780+10%. As we were not so hungry, we decided to go for the 3-course lunch set.Bre

In [164]:
print(text_test_1)

638    It was our second visit. Our first visit about...
705    After hanging out with family and friends for ...
766    8  Otto e Mezzo BOMBANA first opened up in Ale...
771    funny thing is a few weeks before eating at ot...
799    This was our second visit to the 3-starred 8 1...
859    Set lunch: Ham with Autumn SaladPumpkin Raviol...
Name: Reviews, dtype: object


In [158]:
print(text_test_2)

["It was our second visit. Our first visit about a few years ago hadn't given us a very impressive impression but we had just ordered a few items from their a la carte menu so we thought maybe it was a time for a re-visit and for trying their set lunch menu.A window-side table had been reserved quite a while ago during weekday lunch.Lunch Menu1Lunch MenuLunch Menu0Lunch MenuLunch Menu0Lunch MenuLunch Menu0Lunch MenuIt seemed that after a few years their a la carte menu looked pretty much the same. I still remembered that we had tried Marinated Tuna Tartare which was quite good while their Baby Lamb from Aveyron and Short Rib & Beef Tenderloin were nothing outstanding especially the former.Set Lunch Menu1Set Lunch MenuThere were two lunch sets: 4 courses including appetizer, pasta, main and dessert or cheese at $1080+10% and 3 courses with either pasta or main at your choice and everything else the same at $780+10%. As we were not so hungry, we decided to go for the 3-course lunch set.B

In [168]:
# df_all.to_csv("Openrice_trans_all.csv")

In [207]:
df_all_new.to_csv("Openrice_Transformed_V2.csv")